In [1]:
from PIL import Image  # this contain the image class and methods from PIL library
import numpy as np # this imports the numerical and array library 
import cv2
import matplotlib.pyplot as plt
from skimage import measure
import math
import scipy
import os
import sklearn
import glob
from collections import Counter


# Important Note
(If running on Colab)

*   Uncomment and run the pip install commands, as sift is compatible with older versions of openCV
*   You might have to restart the runtime and comment these commands to further run the code



In [2]:
# !pip install opencv-python==3.4.0.14
# !pip install opencv-contrib-python==3.4.2.17

# Change file directory path here
All the results will be stored in cw_data directory.

In [3]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = "/content/drive/MyDrive/Colab Notebooks/cs413/cw_data/DATA/"
try:
  results_dir = "/content/drive/MyDrive/Colab Notebooks/cs413/cw_data/results/"
  os.mkdir(results_dir)
except:
  pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def bounding_box(comps, label=1): # returns bounding boxes of a component in an image
    
    # array of image coordinates in x and y
    xx, yy = np.meshgrid(np.arange(0,comps.shape[1]), np.arange(0,comps.shape[0]))

    # mask/select by where value is given label (component)
    where_x = xx[comps==label]
    where_y = yy[comps==label]
    
    # find min and max extents of coordinates
    return np.min(where_x), np.min(where_y), np.max(where_x), np.max(where_y)

In [5]:
# Same function as Task-1
# Returns a list of all the cards in a training image

def image_extracter(comps,im_gray,im):

  unique = sorted([i[0] for i in Counter(list(comps.ravel())).most_common(25)])# unique labels 

  min_size = 100000 # minimum pixel size threshold
  thres_size = 500000 # Average max pixel size of a card
  max_size = 1500000 # Max Average pixel size of any component.
  bounding_boxes = []
  component_images = [] # list of component images

  for l in unique[1:]: # we know label 0 is always the entire image, so, sliced the list from 1.
      bb = bounding_box(comps, label=l)
      # make a binary image for each component
      one_comp = np.zeros(im.shape[0:2], dtype='uint8')
      one_comp[comps==l] = 1


      # measure its size
      n = np.count_nonzero(one_comp)
      # plot as image if it's big enough
      if (n>min_size) & (n< max_size):
          card_image = []
          if n <= thres_size:

            req_card = im[bb[1]:bb[3],bb[0]:bb[2]]

            #bounding_boxes.append(bb)
            card_image.append(req_card)
            component_images += card_image 
            print('label ', l,'component size is ', n)
            
            plt.imshow(cv2.cvtColor(req_card, cv2.COLOR_BGR2RGB))
            plt.show()
          # Further breaks the components which were left with more than one cards attached, by reducing the threshold further (from 248 to 205)
          # thres_size is used to locate such components which has more than one cards combined.
          else: 
            im_mrthan1 = im_gray[bb[1]:bb[3],bb[0]:bb[2]] 
            img_mrthan1c = im[bb[1]:bb[3],bb[0]:bb[2]]
            threshed_mr1 = np.zeros(im_mrthan1.shape, 'int')
            threshed_mr1[(im_mrthan1<205)] = 1
            comp_mrt1 = measure.label(threshed_mr1,background=0)
            component_images += image_extracter(comp_mrt1,im_mrthan1,img_mrthan1c) # uses recursion to extracts the cards from those comps.

  return component_images

Train-001 cards extract

In [ ]:
# Importing train-001 image
im = cv2.imread(data_dir + 'train-0'+ str(1).zfill(2) +'.jpg')

im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
fig = plt.figure(figsize=(10,10))
plt.imshow(im_gray, 'gray')
plt.show()

# threshold at 248 given the distribution above 
# All the training images has more or less similar distribution, so used the same as task-1
threshed = np.zeros(im_gray.shape, 'int')
threshed[(im_gray<248)] = 1

# creates componenets
comps = measure.label(threshed, background=0)
#plt.imshow(comps)

train_noId_cards = image_extracter(comps,im_gray,im)

In [ ]:
# importing all the training images and their Ids for NN match
train_images = []
train_ids = []
for folder in glob.glob( results_dir + f"train/*"):
  label = folder.split("/")[-1]
  for img in glob.glob(f"{folder}/*"):
    #if str(img).find("aug") < 0:
    image = cv2.imread(img)
    train_images.append(image)
    train_ids.append(label)
    print(img)
                  

# Nearest Neighbour Match SIFT
Used Lowe's keypoints and a metric derived from it to score the closeness between the images.

Score = (total lowe kp matched between descriptors)/(total key points).

Also returns known images (based on a score threshold) which I call validation set for now.

Bear with it as it might takes 15-20 min to extract the matches for all the images

In [8]:
# returns closest labels, images and scores
def NN(img,train_images,M,Ids):

  kp1, des1 = sift.detectAndCompute(img,None)

  perc_similarity = []

  for img2 in train_images:
    kp2, des2 = sift.detectAndCompute(img2,None)

    matches = flann.knnMatch(des1,des2,k=2) # find matches!
    lowe_kp = 0
    for m,n in matches:
      if m.distance < 0.7*n.distance:
        lowe_kp += 1
    perc_similarity.append(lowe_kp/len(kp1)) # Score

  indices = np.argsort(-1*np.array(perc_similarity)) # Unlike distances it has to be sorted in descending order.
  closest_images = [train_images[k] for k in indices]
  closest_perc = [perc_similarity[k] for k in indices]
  labl = [Ids[k] for k in indices]
  return closest_images[:M],closest_perc[:M],labl[:M]


In [26]:
# Uses above written NN function to fetch all the NN images and their labels and score from the training sets and plot them
# Also returns all the known images based on the threshold observed from the data
def match_finder(cards,train_images,train_ids,no_NN):

  known_im = []
  known_im_ids = []
  tot_cards = len(cards)

  for im in cards:
    closest_images,closest_sim_perc,label = NN(im,train_images,no_NN,train_ids)

    if closest_sim_perc[0] > 0.2: # threshold...
    #if first image has threshold > 0.2, called it known image(its present in training data(002-014) as well), validated manually.
      known_im.append(im)
      known_im_ids.append(label[0])
    print("image")
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.show()
    print("matches")
    # display closest images and their distances
    fig = plt.figure(figsize=(20,50))
    for j in range(no_NN):
      plt.subplot(tot_cards,no_NN,j+1)
      plt.imshow(cv2.cvtColor(closest_images[j], cv2.COLOR_BGR2RGB))
      plt.axis('off')
      plt.title(str(round(closest_sim_perc[j],4)))
      plt.subplots_adjust(wspace=None, hspace=None)
        
    plt.show()
  return known_im,known_im_ids 

In [27]:
sift = cv2.xfeatures2d.SIFT_create()
FLANN_INDEX_KDTREE = 1
val_im = []
val_ids = []
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params) # make FLANN searcher
no_NN = 3 # number of nearest neighbors 
known_valc,known_valc_ids = match_finder(train_noId_cards,train_images,train_ids,no_NN)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# creating the val directory
try:
  os.mkdir(results_dir + "val/")
except:
  pass 

In [ ]:
# Saving all the known images in the same format as training dataset
for i in range(len(known_valc_ids)): 
  try:
    os.mkdir(results_dir + f'val/{known_valc_ids[i]}')
  except:
    pass
  cv2.imwrite( results_dir + f'val/{known_valc_ids[i]}/img_val_{i}.jpg',known_valc[i])

# Test data preparation
Image Extracter test: extracts images by using persecpetive tronsform and the minimum area rectangle contours of a component.\
Finds source and destination cordinates using bounding reactangle with minimum area.\
** Minimum area rectangle was easy locate on contours of the components.








In [ ]:
def image_extracter_test(comps,im_gray,im):
  unique = sorted([i[0] for i in Counter(list(comps.ravel())).most_common(20)])# unique labels 

  min_size = 100000 # minimum pixel size threshold
  thres_size = 500000 # Average max pixel size of a card
  max_size = 1500000 # Max Average pixel size of any component.
  bounding_boxes = []
  component_images = [] # list of component images

  for l in unique[1:]:
      bb = bounding_box(comps, label=l)
      # make a binary image for each component
      one_comp = np.zeros(im_gray.shape, dtype='uint8')
      one_comp[comps==l] = 1


      # measure its size
      n = np.count_nonzero(one_comp)

      # plot as image if it's big enough
      if (n>min_size) & (n< max_size):
          card_image = []
          if n <= thres_size:
            # find contours of a component
            contours, hierarchy = cv2.findContours(one_comp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
            # filter noisy detection
            for c in contours:
              if cv2.contourArea(c) > 10000: # exclude noises from the contours

                rect = cv2.minAreaRect(c) # finds minimum area rectangle parameters


                center, size, theta = rect
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                width = int(rect[1][0])
                height = int(rect[1][1])
                # utillising rectangle's corners and width and height to get source and destination corners 
                src_corners = box.astype("float32")

                dst_corners = np.array([[0, height-1],
                        [0, 0],
                        [width-1, 0],
                        [width-1, height-1]], dtype="float32")

                # the perspective transformation matrix
                M = cv2.getPerspectiveTransform(src_corners, dst_corners)

                # warps the image component
                warpped_card = cv2.warpPerspective(im, M, (width, height))
                if height > width:
                  req_card = scipy.ndimage.rotate(warpped_card, 0, reshape=True)
                else:
                  req_card = scipy.ndimage.rotate(warpped_card, 90, reshape=True)

                #bounding_boxes.append(bb)
                card_image.append(cv2.cvtColor(req_card, cv2.COLOR_RGB2BGR))
                component_images += card_image 
                print('label ', l,'component size is ', n)
                
                plt.imshow(req_card)
                plt.show()
          else: # recursively extract cards out of components with more than one card
            im_mrthan1 = im_gray[bb[1]:bb[3],bb[0]:bb[2]]
            img_mrthan1 = im[bb[1]:bb[3],bb[0]:bb[2]]
            threshed_mr1 = np.zeros(im_mrthan1.shape, 'int')
            threshed_mr1[(im_mrthan1<205)] = 1
            comp_mrt1 = measure.label(threshed_mr1,background=0)
            component_images += image_extracter(comp_mrt1,im_mrthan1,img_mrthan1)

  return component_images

In [ ]:
# creates a dictionary with key as test image name and values as list of cards in it.
# Takes around 3-4 minutes to get all the cards
test_images = {}

for folder in glob.glob( data_dir + "*"):
    if str(folder).find("test") >= 0:
      
      image = Image.open(folder)
      im = np.asarray(image)

      im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
      fig = plt.figure(figsize=(10,10))
      plt.imshow(im_gray, 'gray')
      plt.show()

      # masks the green background
      threshed = np.zeros(im_gray.shape, 'int')
      threshed[(im[:,:,1] < 50) | (im[:,:,2] >= im[:,:,1]) | (im[:,:,0] >= im[:,:,1])] = 1

      # creates componenets
      comps = measure.label(threshed, background=0)
      plt.imshow(threshed)
      plt.show

      test_cards = image_extracter_test(comps,im_gray,im)
      test_images[folder.split("/")[-1].split(".")[0]] = test_cards
      #known_testc,known_testc_ids = match_finder(test_cards,train_images,train_ids,no_NN)

In [ ]:
# manually labelled the test datasets with a dictionary (image names : ids)
# Unknown cards are labelled as "000"
test_labels = {"test-001": "000,044,114,109,189,056,138,147,000,154,189,139".split(","),
"test-007": "139,181,138,000,000,134,189,189,147,154,000,000,044,109,056,138".split(","),
"test-010": "109,000,151,134,168,000,168,000,138,154".split(","),
"test-006": "076,172,037,000,001,000,044,114,189,109,138,056,147,000,138".split(","),
"test-009": "181,139,000,134,000,189,154,147,168,000,056,138,044,109".split(","),
"test-005": "001,172,056,109,154,076,044,189,147,000,063,000,189,139,037".split(","),
"test-004": "134,138,000,000,015,154,056,189,114,147,109".split(","),
"test-003": "000,134,000,181,168,189,000,154,147,138,056,109".split(","),
"test-002": "189,000,000,139,044,147,181,109,001,189".split(",")}

In [ ]:
try:
  unique_labels = set([j for i in test_labels.values() for j in i])
  os.mkdir(results_dir + 'test/')
  for cat in unique_labels:
    os.mkdir(results_dir + f'test/{cat}/')
except:
  pass

In [ ]:
test_files = [] # list of all test image names
for folder in glob.glob( data_dir + f"*" ):
  if str(folder).find("test") >= 0:
    test_files.append(folder.split("/")[-1].split(".")[0])

In [ ]:
# Saving the test cards in the same fashion as train and val

for i in test_files:
#os.mkdir("content/drive/MyDrive/Colab Notebooks/cs413/cw_data/test/") 
  testcards = test_images[i]
  label_i = test_labels[i]
  
  for j in range(len(testcards)):
      # print(label_i[j])
      # plt.imshow(testcards[j])
      # plt.show()
      cv2.imwrite( results_dir + f'test/{label_i[j]}/img_{i}_{j}.jpg',testcards[j])